In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### importing abuse report data

In [23]:
data = pd.read_csv("https://raw.githubusercontent.com/tderoon/Cybereconomicsgroup4/master/merge_result.csv")
data.head(20)

,Unnamed: 0,id,dateadded,url,url_status,threat,tags,urlhaus_link,reporter,AS_Number
0,0,544165,9/17/2020 10:10,http://115.98.1.103:56034/Mozi.a,offline,malware_download,None,https://urlhaus.abuse.ch/url/544165/,lrz_security,17488
1,1,544163,9/17/2020 10:10,http://115.97.82.185:47119/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544163/,lrz_security,17488
2,2,544164,9/17/2020 10:10,http://115.97.83.180:58110/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544164/,lrz_security,17488
3,3,544162,9/17/2020 10:10,http://115.98.145.51:60363/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544162/,lrz_security,17488
4,4,544161,9/17/2020 10:10,http://115.97.80.186:42284/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544161/,lrz_security,17488
5,5,544160,9/17/2020 10:10,http://115.98.102.230:45653/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544160/,lrz_security,17488
6,6,544159,9/17/2020 10:09,http://115.97.196.195:48233/Mozi.a,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544159/,lrz_security,17488
7,7,544158,9/17/2020 10:09,http://115.97.31.62:41424/Mozi.a,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544158/,lrz_security,17488
8,8,544157,9/17/2020 10:09,http://achar24.com/wp-admin/lm/,online,malware_download,"emotet,heodo",https://urlhaus.abuse.ch/url/544157/,spamhaus,24940
9,9,544155,9/17/2020 10:09,http://115.97.64.174:45095/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544155/,lrz_security,17488


### exploding the data, per malware tag

In [3]:
exploded_data=data.assign(tags=data.tags.str.split(",")).explode('tags')
exploded_data.head()

,Unnamed: 0,id,dateadded,url,url_status,threat,tags,urlhaus_link,reporter,AS_Number
0,0,544165,9/17/2020 10:10,http://115.98.1.103:56034/Mozi.a,offline,malware_download,None,https://urlhaus.abuse.ch/url/544165/,lrz_security,17488
1,1,544163,9/17/2020 10:10,http://115.97.82.185:47119/Mozi.m,online,malware_download,elf,https://urlhaus.abuse.ch/url/544163/,lrz_security,17488
1,1,544163,9/17/2020 10:10,http://115.97.82.185:47119/Mozi.m,online,malware_download,Mozi,https://urlhaus.abuse.ch/url/544163/,lrz_security,17488
2,2,544164,9/17/2020 10:10,http://115.97.83.180:58110/Mozi.m,online,malware_download,elf,https://urlhaus.abuse.ch/url/544164/,lrz_security,17488
2,2,544164,9/17/2020 10:10,http://115.97.83.180:58110/Mozi.m,online,malware_download,Mozi,https://urlhaus.abuse.ch/url/544164/,lrz_security,17488


In [4]:
#the length of exploded data
len(exploded_data)

265196

In [5]:
#the occurrence of tags
exploded_data['tags'].value_counts()

elf        89369
Mozi       53709
32-bit     35483
mips       34233
exe         6727
           ...  
ono74          1
#trojan        1
Dinihou        1
irc            1
Kpot           1
Name: tags, Length: 197, dtype: int64

### importing the reactspeed per country data

In [12]:
reactspeed = pd.read_csv("https://raw.githubusercontent.com/tderoon/Cybereconomicsgroup4/master/reactspeedURLhaus_inminutes.csv")
reactspeed.head()

,rank,AS_Number,Name,Country,Online,Offline,months,days,hours,minutes,total_in_minutes
0,1,52715,SCORPION TELECOMUNICACAO RIBEIRAO PR ...,BR,0,1,0,0,0,3,3
1,2,13022,STREAMS_GMBH,AT,0,1,0,0,0,5,5
2,3,22438,CLEAR-RATE-COMMUNICATIONS,US,0,1,0,0,0,7,7
3,4,137344,IDNIC-TRANSTEKNO-AS-ID PT. TRANS NAS ...,ID,0,1,0,0,0,7,7
4,5,17716,NTU-TW National Taiwan University,TW,0,1,0,0,0,8,8


### merging the abuse report with the reactspeed data, to get the country of each report

In [30]:
data_withcountry=pd.merge(data,reactspeed,on="AS_Number",how="left")
data_withcountry.head()

,Unnamed: 0,id,dateadded,url,url_status,threat,tags,urlhaus_link,reporter,AS_Number,rank,Name,Country,Online,Offline,months,days,hours,minutes,total_in_minutes
0,0,544165,9/17/2020 10:10,http://115.98.1.103:56034/Mozi.a,offline,malware_download,None,https://urlhaus.abuse.ch/url/544165/,lrz_security,17488,419.0,HATHWAY-NET-AP Hathway IP Over Cable ...,IN,1172.0,73269.0,0.0,0.0,4.0,44.0,284.0
1,1,544163,9/17/2020 10:10,http://115.97.82.185:47119/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544163/,lrz_security,17488,419.0,HATHWAY-NET-AP Hathway IP Over Cable ...,IN,1172.0,73269.0,0.0,0.0,4.0,44.0,284.0
2,2,544164,9/17/2020 10:10,http://115.97.83.180:58110/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544164/,lrz_security,17488,419.0,HATHWAY-NET-AP Hathway IP Over Cable ...,IN,1172.0,73269.0,0.0,0.0,4.0,44.0,284.0
3,3,544162,9/17/2020 10:10,http://115.98.145.51:60363/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544162/,lrz_security,17488,419.0,HATHWAY-NET-AP Hathway IP Over Cable ...,IN,1172.0,73269.0,0.0,0.0,4.0,44.0,284.0
4,4,544161,9/17/2020 10:10,http://115.97.80.186:42284/Mozi.m,online,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/544161/,lrz_security,17488,419.0,HATHWAY-NET-AP Hathway IP Over Cable ...,IN,1172.0,73269.0,0.0,0.0,4.0,44.0,284.0


checking the occurrence of attack per country (not normalized yet!!!)

In [21]:
data_withcountry['Country'].value_counts().head(30)

IN    65268
CN     8117
DE     4909
US     3072
BR      642
CO      616
GE      508
KR      481
RU      400
FR      228
UA      206
TR      202
CA      186
TH      177
EE      160
JP      158
TW      124
GB      113
NL      105
IT       97
VN       92
ES       85
LR       79
ID       67
AU       65
CL       65
AR       58
PL       49
IR       48
MY       43
Name: Country, dtype: int64